# Import all Libraries

In [1]:
from datetime import datetime
import os 
import csv
import time
import requests
import pandas as pd
import tweepy

# Access Twitter API

In [2]:
consumer_key = "RLxjVtPK8BpooxpLw5eMc83rE" 
consumer_secret = "Fh9PYNdzo5r3XBhf2zdp024AFxdgf6QC2tIfXpoRklOC0hlXI2"
access_key = "2170385959-ul1qP90wSnR3wK66AGcmBOmXtCCtpy5pc0U7pTH"
access_secret = "ArtgHqPKFqAgH7re5nDu8isbIqLJKEEgx8HlbOi1EKNnE"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAP83KQEAAAAAIPO3olSWrzwclEphyXk14nbU0Qo%3DyzS6EMakjI0S03DyzFHpHRvp74YufOf3SJbwhNGK7rMP3JBSf3"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

# Scrape Tweets from MMDA Twitter Account
- Open CSV File
- Get the earliest tweet_id
- Collect Tweets
- Add new Tweets to CSV File

In [3]:
def scrape_tweets(username, search_words, count):

    try:
        tweets = tweepy.Cursor(api.search, q=search_words, tweet_mode="extended").items(count)

        tweets_list = [[tweet.id_str, tweet.user.name, tweet.user.verified, tweet.created_at, tweet.full_text] for tweet in tweets if tweet.user.verified and tweet.user.name == username]
    
        tweets_df = pd.DataFrame(tweets_list)
        tweets_df.columns = ['tweet_id','username','verified','date','tweet_content']

    except BaseException as e:
        print('Failed on status,',str(e))
        time.sleep(3)
        
    return tweets_df

In [4]:
def scrape_premium(username, search_words, date_to, count):

    try:
        tweets=tweepy.Cursor(api.search_full_archive,environment_name='Scraper', query=search_words, toDate=date_to).items(count)

        tweets_list = [[tweet.id_str, tweet.user.name, tweet.user.verified, tweet.created_at, tweet.text] for tweet in tweets]
    
        tweets_df = pd.DataFrame(tweets_list)
        tweets_df.columns = ['tweet_id','username','verified','date','tweet_content']

    except BaseException as e:
        print('Failed on status,',str(e))
        time.sleep(3)
        
    return tweets_df

In [5]:
MAIN_FILE = 'MMDA_Tweets.csv'

try:
    mmda_df = pd.read_csv(MAIN_FILE,usecols=[1,2,3,4,5])
    id_past = datetime.strptime(mmda_df['date'].min(),"%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M")
    print(id_past)
except:
    mmda_df = None
    print(MAIN_FILE + " is missing")

MMDA_Tweets.csv is missing


In [6]:
username = 'Official MMDA'
search_word = '"MMDA ALERT:" from:MMDA'
count = 100

In [7]:
tweet_df = scrape_tweets(username, search_word, count)
merge_df = pd.concat([mmda_df,tweet_df])
merge_df.to_csv(MAIN_FILE)

In [ ]:
#---------TEST---------#
tweet_d_df = scrape_premium(username, search_word, id_past, count)
merge_df = pd.concat([mmda_df,tweet_d_df])
merge_df.to_csv(MAIN_FILE)